In [ ]:
# | default_exp patch.full_deps

In [ ]:
# | export
import jax.numpy as jnp
from jaxtyping import Float, Array

## Monkey patching

In [ ]:
# | export
# log weights monkey patch
# nansum to account for missing values
# explicitly assumes that s_t = z_t!

from isssm.util import MVN_degenerate as MVN
import isssm.importance_sampling


def log_weights_t(
    s_t: Float[Array, "p"],  # signal
    y_t: Float[Array, "p"],  # observation
    xi_t: Float[Array, "p"],  # parameters
    dist,  # observation distribution
    z_t: Float[Array, "p"],  # synthetic observation
    Omega_t: Float[Array, "p p"],  # synthetic observation covariance, assumed diagonal
) -> Float:  # single log weight
    """Log weight for a single time point."""
    p_ys = dist(s_t, xi_t).log_prob(y_t).sum()

    # omega_t = jnp.sqrt(jnp.diag(Omega_t))
    # g_zs = MVN_diag(s_t, omega_t).log_prob(z_t).sum()
    g_zs = jnp.nansum(MVN(s_t, Omega_t).log_prob(z_t))

    return p_ys - g_zs


def log_weights(
    s: Float[Array, "n+1 p"],  # signals
    y: Float[Array, "n+1 p"],  # observations
    dist,  # observation distribution
    xi: Float[Array, "n+1 p"],  # observation parameters
    z: Float[Array, "n+1 p"],  # synthetic observations
    Omega: Float[Array, "n+1 p p"],  # synthetic observation covariances:
) -> Float:  # log weights
    """Log weights for all time points"""
    p_ys = dist(s, xi).log_prob(y).sum()

    # avoid triangular solve problems
    # omega = jnp.sqrt(vmap(jnp.diag)(Omega))
    # g_zs = MVN_diag(s, omega).log_prob(z).sum()
    g_zs = jnp.nansum(MVN(s, Omega).log_prob(z))

    return p_ys - g_zs


isssm.importance_sampling.log_weights = log_weights
isssm.importance_sampling.log_weights_t = log_weights_t

In [ ]:
# | export
# LA monkey patch
from isssm.kalman import kalman, smoothed_signals
from isssm.typing import GLSSM, GLSSMProposal, ConvergenceInformation, PGSSM
from isssm.laplace_approximation import default_link, vvmap, vdiag
from jax.scipy.optimize import minimize
from functools import partial
from isssm.util import converged
from jax import jacfwd, hessian, jacrev, vmap, jit
from jax.lax import while_loop
import isssm.laplace_approximation
from isssm.laplace_approximation import _initial_guess, default_link

In [ ]:
def pgsssm_to_glssm(model: PGSSM, Omega: Float[Array, "n+1 p p"]) -> GLSSM:
    return GLSSM(
        u=model.u,
        A=model.A,
        D=model.D,
        Sigma0=model.Sigma0,
        Sigma=model.Sigma,
        v=model.v,
        B=model.B,
        Omega=Omega,
    )

In [ ]:
from isssm.typing import to_glssm

def clip_negative_evals(proposal: GLSSMProposal) -> GLSSMProposal:
    z = proposal.z
    Omega = proposal.Omega

    glssm = to_glssm(proposal)
    filtered = kalman(z, glssm)
    s = smoothed_signals(filtered, z, glssm)
    # clip eigenvalues of final Omega to ensure PSD
    evals, evecs = jnp.linalg.eigh(Omega)
    if (evals > 0).all():
        return proposal

    evals_clipped = jnp.where(evals < 1e-8, 0.0, evals)
    Omega_new = evecs @ vmap(jnp.diag)(evals_clipped) @ jnp.transpose(evecs, (0, 2,1))

    # adjust z s.t. (z -s) lies in span of cOmega, project to span of cOmega, then substract
    cOmega = jnp.linalg.cholesky(Omega_new)
    # projection matrix on im(cOmega ) is cOmega @ jnp.linalg.pinv(cOmega), but use more stable lstsq
    z_new = s + (cOmega @ jnp.linalg.lstsq(cOmega, z- s)[..., None])[...,0]

    new_proposal = GLSSMProposal(
        **proposal,
        z=z_new,
        Omega=Omega,
    )

    return new_proposal

In [ ]:
#| export
def default_link(y):
    return jnp.log(jnp.clip(y, 0.05))

def laplace_approximation(
    y: Float[Array, "n+1 p"],  # observation
    model: PGSSM,
    n_iter: int,  # number of iterations
    log_lik=None,  # log likelihood function
    d_log_lik=None,  # derivative of log likelihood function
    dd_log_lik=None,  # second derivative of log likelihood function
    eps: Float = 1e-5,  # precision of iterations
    link=default_link,  # default link to use in initial guess
) -> tuple[GLSSMProposal, ConvergenceInformation]:
    u, A, D, Sigma0, Sigma, v, B, dist, xi = model
    np1, p, m = B.shape

    s_init = vmap(partial(_initial_guess, dist=dist, link=link))(xi, y)
    # s_init = jnp.zeros((np1, p))  # initial guess for the signal
    # missing values have zero obs. -> 0.
    # s_init = jnp.hstack(
    #    (jnp.log(y.sum(axis=1, keepdims=True) + 1.0), jnp.zeros((np1, 3)))
    # )

    def default_log_lik(
        s_t: Float[Array, "p"], xi_t: Float[Array, "p"], y_t: Float[Array, "p'"]
    ):
        return dist(s_t, xi_t).log_prob(y_t).sum()

    if log_lik is None:
        log_lik = default_log_lik

    if d_log_lik is None:
        d_log_lik = jacfwd(log_lik, argnums=0)
    if dd_log_lik is None:
        dd_log_lik = jacrev(d_log_lik, argnums=0)

    vd_log_lik = jit(vmap(d_log_lik))
    vdd_log_lik = jit(vmap(dd_log_lik))

    def _break(val):
        _, i, z, Omega, z_old, Omega_old = val

        z_converged = jnp.logical_and(converged(z, z_old, eps), i > 0)
        Omega_converged = jnp.logical_and(converged(Omega, Omega_old, eps), i > 0)
        all_converged = jnp.logical_and(z_converged, Omega_converged)

        iteration_limit_reached = i >= n_iter

        return jnp.logical_or(all_converged, iteration_limit_reached)

    def _iteration(val):
        s, i, z_old, Omega_old, _, _ = val

        grad = vd_log_lik(s, xi, y)
        Gamma = -vdd_log_lik(s, xi, y)
        # pinv for missing values: if Gamma is 0, we want Omega to be 0 as well
        # requries exact derivatives, not numerical ones!
        Omega = jnp.linalg.pinv(Gamma, hermitian=True, rcond=1e-10)

        # z = s + jnp.linalg.solve(Gamma, grad[..., None])[..., 0]
        z = s + (Omega @ grad[..., None])[..., 0]
        approx_glssm = GLSSM(u, A, D, Sigma0, Sigma, v, B, Omega)

        filtered = kalman(z, approx_glssm)
        s_new = smoothed_signals(filtered, z, approx_glssm)

        return s_new, i + 1, z, Omega, z_old, Omega_old

    empty_z = jnp.empty_like(s_init)
    empty_Omega = jnp.empty((np1, p, p))
    init = (s_init, 0, empty_z, empty_Omega, empty_z, empty_Omega)

    _keep_going = lambda *args: jnp.logical_not(_break(*args))
    s, n_iters, z, Omega, z_old, Omega_old = while_loop(_keep_going, _iteration, init)

    delta_z = jnp.max(jnp.abs(z - z_old))
    delta_Omega = jnp.max(jnp.abs(Omega - Omega_old))
    information = ConvergenceInformation(
        converged=jnp.logical_and(
            converged(z, z_old, eps), converged(Omega, Omega_old, eps)
        ),
        n_iter=n_iters,
        delta=jnp.max(jnp.array([delta_z, delta_Omega])),
    )

    final_proposal = GLSSMProposal(u, A, D, Sigma0, Sigma, v, B, Omega, z)

    return final_proposal, information


isssm.laplace_approximation.default_link = default_link
isssm.laplace_approximation._initial_guess = _initial_guess
isssm.laplace_approximation.laplace_approximation = laplace_approximation

In [ ]:
# | export
# MEIS monkey patch
import jax.random as jrn
from jaxtyping import PRNGKeyArray
from isssm.importance_sampling import normalize_weights, log_weights_t
from isssm.kalman import simulation_smoother
import isssm.modified_efficient_importance_sampling


def triu(entries: Float[Array, " q"], p: int):
    (q,) = entries.shape

    if 2 * q != p * (p + 1):
        raise ValueError(
            f"entries does not have the correct shape, expected {int(p * (p - 1) /2)} entries in the second dimension but got {q}"
        )

    mat = jnp.zeros((p, p))
    mat = mat.at[jnp.triu_indices(p)].set(entries)

    return mat


def tril(entries: Float[Array, " q"], p: int):
    (q,) = entries.shape

    if 2 * q != p * (p + 1):
        raise ValueError(
            f"entries does not have the correct shape, expected {int(p * (p - 1) /2)} entries in the second dimension but got {q}"
        )

    mat = jnp.zeros((p, p))
    mat = mat.at[jnp.tril_indices(p)].set(entries)

    return mat


@jit
def optimal_parameters(
    signal: Float[Array, "N p"], weights: Float[Array, "N"], log_p: Float[Array, "N"]
):
    N, p = signal.shape
    ones = jnp.ones_like(weights)[:, None]

    second_moment = vmap(jnp.outer)(signal, signal)
    # double tranpose, because [:, jnp.tril_indices(p)] is not supported (returns (N, 2, p(p-1)/2, 4) instead of (N, p(p-1)/2)
    lower_tri_second_moment = second_moment.transpose((1, 2, 0))[jnp.tril_indices(p)].T

    # w_inner_prod = lambda a, b: jnp.einsum("i,ij,ik->jk", weights, a, b)
    # X_T_W_X = jnp.block(
    #    [
    #        [
    #            w_inner_prod(ones, ones),
    #            w_inner_prod(ones, signal),
    #            w_inner_prod(ones, -0.5 * lower_tri_second_moment),
    #        ],
    #        [
    #            w_inner_prod(signal, ones),
    #            w_inner_prod(signal, signal),
    #            w_inner_prod(signal, -0.5 * lower_tri_second_moment),
    #        ],
    #        [
    #            w_inner_prod(-0.5 * lower_tri_second_moment, ones),
    #            w_inner_prod(-0.5 * lower_tri_second_moment, signal),
    #            w_inner_prod(
    #                -0.5 * lower_tri_second_moment, -0.5 * lower_tri_second_moment
    #            ),
    #        ],
    #    ]
    # )
    # X_T_W_y = jnp.concatenate(
    #    [
    #        w_inner_prod(ones, log_p[:, None]),
    #        w_inner_prod(signal, log_p[:, None]),
    #        w_inner_prod(-0.5 * lower_tri_second_moment, log_p[:, None]),
    #    ]
    # )

    design = jnp.hstack([ones, signal, -0.5 * lower_tri_second_moment])
    lhs = log_p

    sqrt_weights = jnp.sqrt(weights)

    design_W = design * sqrt_weights[:, None]
    lhs_W = lhs * sqrt_weights

    # if lhs_W is 0, this returns 0 even if design matrix does not have full rank (numpy promises solution with least l2-norm)
    beta, *_ = jnp.linalg.lstsq(design_W, lhs_W)
    return beta
    # beta = jnp.linalg.solve(X_T_W_X, X_T_W_y[:, 0])
    # return beta


def modified_efficient_importance_sampling(
    y: Float[Array, "n+1 p"],  # observations
    model: PGSSM,  # model
    z_init: Float[Array, "n+1 p"],  # initial z estimate
    Omega_init: Float[Array, "n+1 p p"],  # initial Omega estimate
    n_iter: int,  # number of iterations
    N: int,  # number of samples
    key: PRNGKeyArray,  # random key
    eps: Float = 1e-5,  # convergence threshold
):
    z, Omega = z_init, Omega_init

    np1, p, m = model.B.shape

    key, crn_key = jrn.split(key)

    v_norm_w = vmap(normalize_weights)
    dist = model.dist
    lw_t = vmap(
        vmap(lambda s, y, xi, z, Omega: log_weights_t(s, y, xi, dist, z, Omega)),
        (0, None, None, None, None),
    )

    def _break(val):
        i, z, Omega, z_old, Omega_old = val

        # in first iteration we don't have old values, converged is True for NaNs
        z_converged = jnp.logical_and(converged(z, z_old, eps), i > 0)
        Omega_converged = jnp.logical_and(converged(Omega, Omega_old, eps), i > 0)
        iteration_limit_reached = i >= n_iter

        return jnp.logical_or(
            jnp.logical_and(z_converged, Omega_converged), iteration_limit_reached
        )

    def _iteration(val):
        i, z, Omega, _, _ = val
        glssm_approx = GLSSM(
            model.u,
            model.A,
            model.D,
            model.Sigma0,
            model.Sigma,
            model.v,
            model.B,
            Omega,
        )
        sim_signal = simulation_smoother(glssm_approx, z, N, crn_key)

        log_weights = lw_t(sim_signal, y, model.xi, z, Omega)
        log_p = dist(sim_signal, model.xi).log_prob(y).sum(axis=-1)
        wls_estimate = vmap(optimal_parameters, (1, 1, 1), 0)(
            sim_signal, v_norm_w(log_weights), log_p
        )

        a = wls_estimate[:, 0]
        b = wls_estimate[:, 1 : (p + 1)]
        c = wls_estimate[:, (p + 1) :]

        Omega_inv_new = vmap(tril, (0, None))(c, p)
        Omega_inv_new = 1 / 2 * (Omega_inv_new + Omega_inv_new.transpose((0, 2, 1)))
        # pinv for missing cases
        Omega_new = jnp.linalg.pinv(Omega_inv_new, hermitian=True)
        z_new = (Omega_new @ b[..., None])[..., 0]

        return i + 1, z_new, Omega_new, z, Omega

    _keep_going = lambda *args: jnp.logical_not(_break(*args))

    init = (0, z_init, Omega_init, jnp.empty_like(z_init), jnp.empty_like(Omega_init))
    n_iters, z, Omega, z_old, Omega_old = while_loop(_keep_going, _iteration, init)

    proposal = GLSSMProposal(
        model.u, model.A, model.D, model.Sigma0, model.Sigma, model.v, model.B, Omega, z
    )

    delta_z = jnp.max(jnp.abs(z - z_old))
    delta_Omega = jnp.max(jnp.abs(Omega - Omega_old))
    information = ConvergenceInformation(
        converged=jnp.logical_and(
            converged(z, z_old, eps), converged(Omega, Omega_old, eps)
        ),
        n_iter=n_iters,
        delta=jnp.max(jnp.array([delta_z, delta_Omega])),
    )

    return proposal, information


isssm.modified_efficient_importance_sampling.modified_efficient_importance_sampling = (
    modified_efficient_importance_sampling
)

In [ ]:
#| export
# monkey patch gnll to handle nans
vmm = vmap(jnp.matmul, (0,0))
from isssm.util import MVN_degenerate as MVN
import isssm.estimation
@jit
def gnll(
    y: Float[Array, "n+1 p"],  # observations $y_t$
    x_pred: Float[Array, "n+1 m"],  # predicted states $\hat X_{t+1\bar t}$
    Xi_pred: Float[Array, "n+1 m m"],  # predicted state covariances $\Xi_{t+1\bar t}$
    B: Float[Array, "n+1 p m"],  # state observation matrices $B_{t}$
    Omega: Float[Array, "n+1 p p"],  # observation covariances $\Omega_{t}$
) -> Float:  # gaussian negative log-likelihood
    """Gaussian negative log-likelihood"""
    y_pred = vmm(B, x_pred)
    Psi_pred = vmm(vmm(B, Xi_pred), jnp.transpose(B, (0, 2, 1))) + Omega

    return jnp.nansum(-MVN(y_pred, Psi_pred).log_prob(y))

isssm.estimation.gnll = gnll

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()